![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=Health/CALM/CALM-moving-out-2.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# CALM - Moving Out Project
## Part 2 - Income

📗Now let's take a look at your potential income. First, some definitions:

### Paycheque Definitions
#### Gross Income (pay/earnings)
The amount of income/earnings, for any pay period, before deductions
 
#### Net income (pay/earnings)
The amount of income/earnings, for any pay period, after deductions (Take home pay)

#### CPP - Canada Pension Plan
2.3% of gross income deducted for insurance in case of unemployment
 
#### Income Tax
A deduction paid to the Federal and Provincial government for taxes
 
#### LTD
A deduction for Long Term Disability insurance
 
#### Union Dues
Fees paid for membership in a union
 
#### Bonds
An investment in which a business or government pays a set interest rate
 
#### Advance Earnings
Deducted money that was received in advance of the pay cheque
 
#### Overtime Earnings
Pay received for working over 8 hours a day or 44 hours a week, whichever is greater

### Calculating Net Income

📗Click on the code cell below, then click the `Run` button on the toolbar to calculate your net income. You may also change some values in the code to see how the results change.

In [ ]:
%store -r
import warnings
warnings.filterwarnings("ignore")


In [ ]:
import pandas as pd
dataset = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/Health/CALM/Moving-out-2/income-statistics.csv")
dataset = dataset.rename(columns={'Work activity during the reference year (9)':'Work Type','Occupation - Minor group - National Occupational Classification (NOC) 2021 (309A)':'Occupation','Employment income statistics (7):Average employment income ($)[4]':'Average Income'})

dataset

In [ ]:
remove_occ = ['Total - Occupation - Minor group - National Occupational Classification (NOC) 2021','Occupation - not applicable','All occupations']

dataset = dataset.loc[(dataset['Occupation'] != remove_occ[1]) & (dataset['Occupation'] != remove_occ[2])]
dataset = dataset.loc[(dataset['GEO'] == 'Canada') & (dataset['Average Income'] != 0)]
dataset

In [ ]:
age_group_avg_income = dataset.groupby(['Age (15A)','Statistics (3)'])['Average Income'].mean().reset_index()
age_group_avg_income = a.loc[(a['Statistics (3)'] == 'Count') & ((a['Age (15A)'] != '15 to 19 years') & ((a['Age (15A)'] != '15 to 24 years')) & (a['Age (15A)'] != '25 to 29 years') & (a['Age (15A)'] != '25 to 54 years') & (a['Age (15A)'] != '25 to 64 years') & (a['Age (15A)'] != '65 years and over') & (a['Age (15A)'] != 'Total - Age'))]

age_group_avg_income 

In [ ]:
import plotly.express as px

px.bar(age_group_avg_income,x='Age (15A)',y='Average Income')

In [ ]:
import numpy as np
import pandas as pd

usa = pd.read_excel('https://raw.githubusercontent.com/callysto/data-files/main/Health/CALM/Moving-out-2/occupation-salaries.xslx')

usa_shorten = usa[['OCC_TITLE','H_MEAN','A_MEAN']]

usa_shorten.drop_duplicates(inplace=True)
usa_shorten['H_MEAN'] = usa_shorten['H_MEAN'].replace('*',np.nan)
usa_shorten['A_MEAN'] = usa_shorten['A_MEAN'].replace('*',np.nan)
usa_shorten.dropna(inplace=True)

usa_shorten['H_MEAN'] = usa_shorten['H_MEAN'].astype(float)
usa_shorten['A_MEAN'] = usa_shorten['A_MEAN'].astype(float)
usa_shorten['M_MEAN'] = usa_shorten['A_MEAN']/12

usa_shorten = usa_shorten.tail(-2)
usa_shorten

In [ ]:
converted_salary_min = desired_salary[0]
converted_salary_max = desired_salary[1]

my_possible_careers = usa_shorten.loc[(usa_shorten['M_MEAN'] >= converted_salary_min) & (usa_shorten['M_MEAN'] <= converted_salary_max)]

my_possible_careers.sort_values('A_MEAN',ascending=False,inplace=True)

print("TOP 10 paying jobs based on your desired salary: \n")
for jobs in my_possible_careers.head(10).iterrows():
    print(str(jobs[1]["OCC_TITLE"])  + " - $" + str(jobs[1]['A_MEAN']))

In [ ]:
#✏️
wagePerHour = 15  # this is your wage in $/hour
hoursPerDay = 8
workdaysPerMonth = 21

grossIncome = wagePerHour * hoursPerDay * workdaysPerMonth
print('Your gross income is $', grossIncome, 'per month.')

incomeTax = .15 + .10   # assume federal income tax is 15% and provincial is 10%
cpp = .0495             # assume Canada Pension Plan is 4.95%
ei = .0188              # assume Employment Insurance is 1.88%
unionDues = .0075       # 0.75% sounds reasonable for union dues

deductions = grossIncome * (incomeTax + cpp + ei + unionDues)
print('$', '{:.2f}'.format(deductions), ' will be taken off your paycheck.')

netIncome = grossIncome - deductions
print('Your net income is $', '{:.2f}'.format(netIncome), 'per month.')
# store that value in memory for later notebooks
%store netIncome

## Graphing Income
📗
We can also look at how your net income (take-home pay) changes based on your hourly wage. We will use [2019 taxation rates](https://www.canada.ca/en/revenue-agency/services/tax/individuals/frequently-asked-questions-individuals/canadian-income-tax-rates-individuals-current-previous-years.html) as well as [EI](https://www.canada.ca/en/revenue-agency/services/tax/businesses/topics/payroll/payroll-deductions-contributions/employment-insurance-ei/ei-premium-rates-maximums.html) and [CPP](https://www.canada.ca/en/revenue-agency/services/tax/businesses/topics/payroll/payroll-deductions-contributions/canada-pension-plan-cpp/cpp-contribution-rates-maximums-exemptions.html) rates. `Run` the cell below (without editing it) to generate a graph.
</div>

In [ ]:
#📗
from functions import *

wages = []
grossIncomes = []
netIncomes = []
for wage in range(15, 150):
    wages.append(wage)
    grossAnnualIncome = wage * 8 * 240
    grossIncomes.append(grossAnnualIncome)
    incomeTax = calculateFederalTax(grossAnnualIncome) + calculateProvincialTax(grossAnnualIncome)
    eiPremium = calculateEI(grossAnnualIncome)
    cppPremium = calculateCPP(grossAnnualIncome)
    netIncome = grossAnnualIncome - (incomeTax + eiPremium + cppPremium)
    netIncomes.append(netIncome)

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=wages, y=grossIncomes, name='Gross Income'))
fig.add_trace(go.Scatter(x=wages, y=netIncomes, name='Net Income'))
fig.update_layout(
    title=go.layout.Title(text='Net Income vs Hourly Wage'),
    xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Hourly Wage')),
    yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Income')))
fig.show()

📗The graph shows that the difference between gross income and net income (after deductions) increases as wage increases. For more information about this, you may want to read about [progressive taxation](https://en.wikipedia.org/wiki/Progressive_tax).

You have now completed this section. Proceed to [section 3](./CALM-moving-out-3.ipynb)

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)